In [5]:
import os
import openai
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import random
import pandas as pd
import re
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0)

In [ ]:
import re
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Meta-Prompt Definition
meta_prompt = PromptTemplate(
    input_variables=["original_prompt"],
    template="""
### SYSTEM MESSAGE
You are a highly advanced AI specializing in **prompt enhancement**.
Your goal is to take an input prompt and **optimize it into a clear, structured, and highly effective instruction.**
You will follow a **multi-step reasoning process** before providing the final enhanced version.

---

### STEP 1️⃣: THINK BEFORE RESPONDING (ReAct)
**Before improving the prompt, analyze it carefully. Answer these questions first:**
1. **Missing Details:** What critical information is missing from this prompt?
2. **Intent Clarity:** What is the user’s intended goal with this prompt?
3. **Ideal Format:** What structure should the improved prompt follow?
4. **Target Audience:** Who is the audience for this response? (e.g., beginner, expert, general)
5. **Best Wording Choices:** Which keywords or phrases would improve specificity?

---

### STEP 2️⃣: GENERATE MULTIPLE VARIANTS (Self-Consistency)
**Generate three different versions** of the improved prompt, each with a unique angle:
1. **Concise & Direct Version** (Short but highly effective)
2. **Highly Detailed Version** (Expanded with full clarity and structure)
3. **Creative & Engaging Version** (Adds storytelling or engaging elements)

---

### STEP 3️⃣: SCORE & SELECT THE BEST PROMPT
For each of the three generated prompts, **evaluate** them based on:
- ✅ **Clarity (0-10):** Is the improved prompt clear and unambiguous?
- ✅ **Detail (0-10):** Does it contain all necessary specifics?
- ✅ **Effectiveness (0-10):** How useful is the improved prompt for achieving the user’s goal?

💡 **Select the highest-scoring prompt** and return it as the final output.

---

### INPUT
**Original Prompt:** "{original_prompt}"

---

### OUTPUT
💡 **Thought Process (ReAct):**
(Missing details, intent, ideal format, audience, best wording choices)

✍ **Generated Variants (Self-Consistency):**
1️⃣ **Concise Version:** ...
2️⃣ **Detailed Version:** ...
3️⃣ **Creative Version:** ...

🏆 **Best Version Selected (Highest Score):** 
"""
)

def enhance_prompt(original_prompt):
    response = llm.invoke(meta_prompt.format(original_prompt=original_prompt)).content
    
    best_prompt_match = re.search(
        r"🏆 \*\*Best Version Selected.*?\*\*:\s*(.+)", 
        response,
        re.DOTALL 
    )
    
    if best_prompt_match:
        return best_prompt_match.group(1).strip() 
    else:
        print("🔴 Warning: Best prompt not found. Returning full LLM response for debugging.")
        return response 
improved_prompt = enhance_prompt("Analysis of the top trending Ethereum Projects.")
print("\n✅ **Final Optimized Prompt:**\n", improved_prompt)

🔴 Warning: Best prompt not found. Returning full LLM response for debugging.

✅ **Final Optimized Prompt:**
 💡 **Thought Process (ReAct):**

1. **Missing Details:**  
   - The original prompt lacks specifics about the type of analysis required (e.g., technical, financial, community engagement).  
   - It does not specify the time frame for "top trending" (e.g., past week, month, year).  
   - The audience is unclear—are they beginners, investors, developers, or researchers?  
   - It does not clarify the format of the output (e.g., report, summary, comparison).  

2. **Intent Clarity:**  
   The user likely wants a comprehensive analysis of Ethereum projects that are currently popular, but the exact focus (e.g., market performance, innovation, adoption) is ambiguous.  

3. **Ideal Format:**  
   A structured prompt should include:  
   - The scope of the analysis (e.g., top 5 projects, specific metrics).  
   - The time frame for "trending."  
   - The type of insights desired (e.g., g